In [1]:
import pandas as pd 
import numpy as np 
from itertools import product

In [2]:
countries = ['us','de','dk','fr','it','nl','se','sp']

In [3]:
test = pd.read_pickle('output/params_ref_us.pkl')
test

,value,free,se
name,,,
sigma,1.272475,True,0.114942
beta,0.950000,False,NaN
phi,0.423766,True,0.269067
psi,0.213239,True,0.028317
delta_h1,-0.987410,True,0.015829
delta_h2,3.343512,True,0.041981
eta,0.000000,False,NaN
tfp,1.000000,False,NaN
price,1.000000,False,NaN


In [4]:
pars_select = ['delta_h1','delta_h2','price','tfp']
stat = ['par','se']
tuples = list(product(*[pars_select,stat]))
table = pd.DataFrame(index=pd.MultiIndex.from_tuples(tuples),columns=countries)
for c in countries: 
	df = pd.read_pickle('output/params_ref_'+c+'.pkl')
	for p in pars_select:
		table.loc[(p,'par'),c] = df.loc[p,'value']
		table.loc[(p,'se'),c] = df.loc[p,'se']


In [5]:
pars_select = ['delta_h1','delta_h2','price','tfp']
labels = ['$\\alpha_{10}$','$\\alpha_{11}$','$\\frac{p}{p_{US}}$','$\\frac{A}{A_{US}}$']
maps_labels = dict(zip(pars_select,labels))
table.index = pd.MultiIndex.from_tuples(list(product(*[labels,['','se']])))

In [6]:
for c in table.columns:
	table[c] = table[c].astype('float64')
	table[c] = table[c].round(3)
	table[c] = table[c].astype('str')
	table[c] = np.where(table[c]=='nan','-',table[c])
	for s in table.index:
		if s[1]=='se':
			if table.loc[s,c]!='-':
				fmt = "({se})"
				table.loc[s,c] = fmt.format(se=table.loc[s,c])
table.index = pd.MultiIndex.from_tuples(list(product(*[labels,['','']])))

In [7]:
table

us       de       dk       fr       it       nl  \
$\alpha_{10}$         -0.987   -1.281   -1.773    -1.17    -0.84    -1.59   
                     (0.016)  (0.047)   (0.02)  (0.085)  (0.094)   (0.45)   
$\alpha_{11}$          3.344    3.947    4.233    3.991    3.896    3.978   
                     (0.042)  (0.009)  (0.018)  (0.097)  (0.095)  (0.076)   
$\frac{p}{p_{US}}$       1.0     0.84    0.937    0.855    0.676    0.708   
                           -  (0.004)  (0.011)  (0.006)  (0.033)  (0.633)   
$\frac{A}{A_{US}}$       1.0    1.017    1.227    0.931    0.677    1.014   
                           -  (0.022)  (0.009)  (0.007)  (0.249)  (1.167)   

                          se       sp  
$\alpha_{10}$         -1.412    0.134  
                     (0.077)  (0.053)  
$\alpha_{11}$          4.417    3.391  
                     (0.074)  (0.129)  
$\frac{p}{p_{US}}$      0.89    1.085  
                     (0.158)  (0.176)  
$\frac{A}{A_{US}}$     0.801    0.837  
                     (0.011)  (0.052)

In [8]:
table.to_latex('../tables/table_8_country_specific_estimates.tex')

## Average US-EU differences

This number is used in counterfactual analysis. 

In [9]:
table.index = pd.MultiIndex.from_tuples(list(product(*[labels,['p','se']])))
prices = table.loc[('$\\frac{p}{p_{US}}$','p'),:]
prices['de':].astype('float64').mean()

0.8558571428571428

In [10]:
table.index = pd.MultiIndex.from_tuples(list(product(*[labels,['p','se']])))
prices = table.loc[('$\\frac{A}{A_{US}}$','p'),:]
prices['de':].astype('float64').mean()

0.929142857142857